In [1]:
%set_env LC_ALL=en_US.UTF-8
%set_env LANG=en_US.UTF-8
import os
os.environ["LC_ALL"] = "en_US.UTF-8"
os.environ["LANG"] = "en_US.UTF-8"
import nltk
# import MySQLdb
import en_coref_md
import sys

nlp = en_coref_md.load()

env: LC_ALL=en_US.UTF-8
env: LANG=en_US.UTF-8


In [ ]:
def line_splitter(text):
    lines_in_article = [s.strip() for s in text.splitlines()]
    return lines_in_article

def print_lines(lines):
    for elem in lines:
        print elem
        print "------------------"

In [ ]:
import subprocess
import os
import sys
import time
from nltk import ngrams
from nltk import word_tokenize


fw = open("tmpout", "wb")
fr = open("tmpout", "r")

proc = subprocess.Popen(['java', "-mx8g", "-cp", "/Users/kumar.jitender/Downloads/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar:/Users/kumar.jitender/Downloads/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0-models.jar:/Users/kumar.jitender/Downloads/stanford-corenlp-full-2017-06-09/CoreNLP-to-HTML.xsl:slf4j-api.jar:/Users/kumar.jitender/Downloads/stanford-corenlp-full-2017-06-09/slf4j-simple.jar", "edu.stanford.nlp.naturalli.OpenIE",  "-threads",  "8", "-ignore_affinity", "true",  "-ssplit.newlineIsSentenceBreak", "always", "-format", "ollie"], stdout=fw, stderr=fw, stdin=subprocess.PIPE)

proc.stdin.write("Milo Blah Blah\n")
proc.stdin.flush()

# Waiting for the model to load properly
time.sleep(20)
# Discarding initial Output
print fr.read()

def parse_oie_op(line):
    if line.find(':') !=-1:
        rel_substring = line[line.find(':') + 3:]
        if rel_substring.find(';') != -1:
            e1 = rel_substring[:rel_substring.find(';')]
            rel_substring = rel_substring[rel_substring.find(';')+2:]
            if rel_substring.find(';') != -1:
                rel = rel_substring[:rel_substring.find(';')]
                rel_substring = rel_substring[rel_substring.find(';')+2:]            
                if rel_substring.find(')') != -1:
                    e2 = rel_substring[:rel_substring.find(')')]
                    return (e1, rel, e2)
                else:
                    raise ValueError(line)
            else:
                raise ValueError(line)
        else:
            raise ValueError(line)
        

def extract_relations(sentence):
    no_of_tokens = len(nltk.word_tokenize(sentence))
    myset=set()
    for iter in range(max(4, no_of_tokens-5), no_of_tokens+1):
        sixgrams = ngrams(sentence.split(), iter)
        new_list = [' '.join(words) for words in sixgrams] 
        for gram in new_list:
            proc.stdin.write( gram + "\n")
            proc.stdin.flush()
        
    time.sleep(10)
    for line in fr.read().splitlines():
        if 'No extractions in' not in line:
            myset.add(parse_oie_op(line))
    return myset

In [ ]:
import nltk
import re
import pprint
from nltk import Tree

# patterns = """
#     NP: {<JJ>*<NN*>+}
#     {<JJ>*<NN*><CC>*<NN*>+}
#     """

patterns = """
    NP:    {<DT><WP><VBP>*<RB>*<VBN><IN><NN>}
           {<NN|NNS|NNP|NNPS><IN>*<NN|NNS|NNP|NNPS>+}
           {<JJ|JJR|JJS>*<NN|NNS|NNP|NNPS><CC>*<NN|NNS|NNP|NNPS>+}
           {<JJ|JJR|JJS>*<NN|NNS|NNP|NNPS>+}
           
    """

NPChunker = nltk.RegexpParser(patterns)

def prepare_text(input):
    sentences = nltk.sent_tokenize(input)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences = [NPChunker.parse(sent) for sent in sentences]
    print(sentences)
    return sentences


def parsed_text_to_NP(sentences):
    nps = []
    for sent in sentences:
        tree = NPChunker.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                t = subtree
                t = ' '.join(word for word, tag in t.leaves())
                nps.append(t)
    return nps


def find_nps(text):
    prepared = prepare_text(text)
    return parsed_text_to_NP(prepared)

In [ ]:
def sortSecond(val):
    return val[1] 

def get_coreferences(paragraph):
    doc = nlp(unicode(paragraph , 'latin-1'))
    lst_replace = []
    if doc._.has_coref:
        for cluster in doc._.coref_clusters:
            for mention in cluster.mentions:
                lst_replace.append((mention.start_char, mention.end_char, mention.text, cluster.main.text))

    lst_replace.sort(key=sortSecond)        
    return lst_replace


def get_coref_replacement_dict(coref_list, sentence_start_char, sentence_end_char):
    relevant_corefs = {}
    for coref in coref_list:
        if coref[0] >= sentence_start_char and coref[1] <= sentence_end_char:
            relevant_corefs[remove_stopwords(coref[2])] = coref[3]
    return relevant_corefs


def prcoess_relations(paragraph):

    #     get all coreferences in paragraph
    coref_list = get_coreferences(paragraph);
#     print "PARAGRAPH"
#     print paragraph
#     print "COREFERENCES"
#     print coref_list
    
    #     Split sentences here
    all_sentences = nltk.sent_tokenize(paragraph)
    
    sentence_start_char = 0
    sentence_end_char = 0
    for sentence in all_sentences:
        print ("-----------------------------")
        print ("SENTENCE")
        print (sentence)

        sentence_start_char = sentence_end_char
        sentence_end_char = sentence_start_char + len(sentence)
        
        #     Run Relationship extraction on sentence via Stanford NLP
        relationships_set = extract_relations(sentence)
        print ("ALL RELATIONSHIPS")
        print (relationships_set)
        
        #     Figure stopword removed coreference replacements for that sentence
        coref_replacement_dict = get_coref_replacement_dict(coref_list, sentence_start_char, sentence_end_char)
        print ("COREF REPLACEMENTS")
        print (coref_replacement_dict)

        #     Remove stopwords from relations
        relationships_set_stopword_remove = set()
        for relation in relationships_set:
            if not remove_stopwords(relation[1]):
                new_relation = (remove_stopwords(relation[0]), relation[1], remove_stopwords(relation[2]))
            else:
                new_relation = (remove_stopwords(relation[0]), remove_stopwords(relation[1]), remove_stopwords(relation[2]))
            relationships_set_stopword_remove.add(new_relation)

        print ("RELATIONSHIPS AFTER REMOVING STOPWORDS")
        print (relationships_set_stopword_remove)
        
            
        #     Replace coreferences
        relationships_set_coref_replace = set()
        for relation in relationships_set_stopword_remove:
            if relation[0] in coref_replacement_dict:
                e1 = coref_replacement_dict[relation[0]]
            else:
                e1 = relation[0]
            if relation[2] in coref_replacement_dict:
                e2 = coref_replacement_dict[relation[2]]
            else:
                e2 = relation[2]
            new_relation = (e1, relation[1], e2)
            relationships_set_coref_replace.add(new_relation)
        
        print ("RELATIONSHIPS AFTER CoREF REPLACEMENT")
        print (relationships_set_coref_replace)
        
        
        #     Figure Nounphrases for that sentence                
        noun_phrases = find_nps(sentence)
        print ("NOUN PHRASES")
        print (noun_phrases)
        
        # Replace corefs in noun_phrases as well
        # Noun phrases are already stopword removed, if not, remove
        
        noun_phrases_coref_filtered = set()
        for noun_phrase in noun_phrases:
            noun_phrase_temp = remove_stopwords(noun_phrase)
            if noun_phrase_temp in coref_replacement_dict:
                noun_phrases_coref_filtered.add(coref_replacement_dict[noun_phrase_temp])
            else:
                noun_phrases_coref_filtered.add(noun_phrase_temp)
                            
        filtered_relationships_set = set()
        #     Filter relations on the NPhrases
        for relation in relationships_set_coref_replace:
            if relation[0] in noun_phrases_coref_filtered and relation[2] in noun_phrases_coref_filtered:
                filtered_relationships_set.add(relation)
                
        print ("RELATIONSHIPS AFTER NOUN PHRASE FIltering")
        print (filtered_relationships_set)
    return filtered_relationships_set

In [ ]:
print(find_nps("Celebrity-favourite label Pumas shoes already come with a firm fan following but throw in Sophia Websters signature, playful aesthetic and youre guaranteed cult status."))

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 

def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence)
    filtered_sentence = []
    
    for w in word_tokens:
        if w.lower() not in stop_words:
            filtered_sentence.append(w)
        
    return ' '.join(filtered_sentence)

In [ ]:
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="foobar",  # your password
                     db="new_trends")        # name of the data base

cur = db.cursor()

# Use all the SQL you like
cur.execute("SELECT * FROM TEXTS WHERE SOURCE = 'DIFFBOT' and type = 'BODY' and id =93")

# print all the first cell of all the rows
for row in cur.fetchall():
    if row[1]:
        lines = line_splitter(row[1])
        for line in lines:
            prcoess_relations(line)
        print "********************************************************************************"

In [ ]:
After Kate Middleton and Meghan Markle stepped out together in navy dresses, Diana Penty made an appearance in a deep blue kurta set day before.